In [110]:
import platform
import sklearn
import pandas as pd
import numpy as np
import os
from __future__ import division

In [111]:
data_file = ("nba_data.csv")

In [112]:
from sklearn.metrics import f1_score, make_scorer, classification_report
scorer = make_scorer(f1_score, pos_label=None, average='weighted')

In [113]:
results = pd.read_csv(data_file, parse_dates=["Date"])
results.rename(columns = {'Visitor/Neutral':'Visitor', 'PTS':'VisitorPts', 'Home/Neutral':'Home', 'PTS.1':'HomePts', 'Unnamed:6':'Score Type', 'Unnamed: 7':'OT?'}, inplace = True)
results["VisitorPts"] = results["VisitorPts"].apply(pd.to_numeric, errors='coerce')
results["HomePts"] = results["HomePts"].apply(pd.to_numeric, errors='coerce')
results["Home Team Win"] = results["HomePts"] > results["VisitorPts"]

In [116]:
num_games = results["Home Team Win"].count()
num_home_wins = results["Home Team Win"].sum()
home_win_percentage =  (num_home_wins/num_games * 100)

In [117]:
results["Home Last Win"] = False
results["Visitor Last Win"] = False

In [121]:
from collections import defaultdict
won_last = defaultdict(int)

for i, r in results.iterrows():
    home_team = r["Home"]
    visitor_team = r["Visitor"]
    r["Home Last Win"] = won_last[home_team]
    r["Visitor Last Win"] = won_last[visitor_team]
    results.ix[i] = r
    won_last[home_team] = r["Home Last Win"]
    won_last[visitor_team] = not r["Home Last Win"]

/Users/TanmayGhai/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':


In [124]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=14)